# Historical Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as plx

## Load Dataset

In [2]:
trades = pd.read_csv(
   '../data/raw/historical_data/historical_data.csv',
    parse_dates=["Timestamp IST"],
    dayfirst=True,
    low_memory=False
)

## Standardize the columns

In [3]:
trades.columns = (
    trades.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace(r"[^\w\s]", "", regex=True)
)

## Strip whitespace in string columns


In [4]:
for col in trades.select_dtypes(include="object"):
    trades[col] = trades[col].astype(str).str.strip()

In [5]:
trades.head()

,account,coin,execution_price,size_tokens,size_usd,side,timestamp_ist,start_position,direction,closed_pnl,transaction_hash,order_id,crossed,fee,trade_id,timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,2024-12-02 22:50:00,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,2024-12-02 22:50:00,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,2024-12-02 22:50:00,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,2024-12-02 22:50:00,1146.558564,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,2024-12-02 22:50:00,1289.488521,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


## Check for Duplicate values

In [6]:
for col in trades.columns:
    print(f"{col} : {trades[col].duplicated().sum()} duplicates")

account : 211192 duplicates
coin : 210978 duplicates
execution_price : 151062 duplicates
size_tokens : 151920 duplicates
size_usd : 92731 duplicates
side : 211222 duplicates
timestamp_ist : 183247 duplicates
start_position : 14301 duplicates
direction : 211212 duplicates
closed_pnl : 120504 duplicates
transaction_hash : 110040 duplicates
order_id : 160669 duplicates
crossed : 211222 duplicates
fee : 72422 duplicates
trade_id : 208414 duplicates
timestamp : 211217 duplicates


## Check for Unique values

In [7]:
for col in trades.columns:
    print(f"{col} : {trades[col].nunique()} uniques")

account : 32 uniques
coin : 246 uniques
execution_price : 60162 uniques
size_tokens : 59304 uniques
size_usd : 118493 uniques
side : 2 uniques
timestamp_ist : 27977 uniques
start_position : 196923 uniques
direction : 12 uniques
closed_pnl : 90720 uniques
transaction_hash : 101184 uniques
order_id : 50555 uniques
crossed : 2 uniques
fee : 138802 uniques
trade_id : 2810 uniques
timestamp : 7 uniques


In [8]:
trades.shape

(211224, 16)

## Convert IST → UTC

In [9]:

trades = trades.rename(columns={"timestamp_ist": "timestamp_local"})
trades["timestamp_local"] = pd.to_datetime(trades["timestamp_local"], dayfirst=True)
trades["timestamp_local"] = trades["timestamp_local"].dt.tz_localize("Asia/Kolkata")
trades["timestamp_utc"] = trades["timestamp_local"].dt.tz_convert("UTC")

## Time features

In [10]:
trades["date"] = trades["timestamp_utc"].dt.date
trades["hour"] = trades["timestamp_utc"].dt.hour
trades["weekday"] = trades["timestamp_utc"].dt.weekday
trades["week"] = trades["timestamp_utc"].dt.isocalendar().week
trades["year"] = trades["timestamp_utc"].dt.year
trades["is_weekend"] = trades["weekday"].isin([5, 6]).astype(int)

## Numeric features

In [11]:
trades["win"] = (trades["closed_pnl"] > 0).astype(int)
trades["abs_size_tokens"] = trades["size_tokens"].abs()
trades["abs_size_usd"] = trades["size_usd"].abs()
trades["roi"] = np.where(
    trades["size_usd"] != 0,
    trades["closed_pnl"] / trades["size_usd"],
    0
)

In [12]:
trades

,account,coin,execution_price,size_tokens,size_usd,side,timestamp_local,start_position,direction,closed_pnl,...,date,hour,weekday,week,year,is_weekend,win,abs_size_tokens,abs_size_usd,roi
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,2024-12-02 22:50:00+05:30,0.000000,Buy,0.0000,...,2024-12-02,17,0,49,2024,0,0,986.87,7872.16,0.000000
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,2024-12-02 22:50:00+05:30,986.524596,Buy,0.0000,...,2024-12-02,17,0,49,2024,0,0,16.00,127.68,0.000000
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,2024-12-02 22:50:00+05:30,1002.518996,Buy,0.0000,...,2024-12-02,17,0,49,2024,0,0,144.09,1150.63,0.000000
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,2024-12-02 22:50:00+05:30,1146.558564,Buy,0.0000,...,2024-12-02,17,0,49,2024,0,0,142.98,1142.04,0.000000
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,2024-12-02 22:50:00+05:30,1289.488521,Buy,0.0000,...,2024-12-02,17,0,49,2024,0,0,8.73,69.75,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211219,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.1010,382.20,420.80,SELL,2025-04-25 15:35:00+05:30,7546.600000,Close Long,-20.2566,...,2025-04-25,10,4,17,2025,0,0,382.20,420.80,-0.048138
211220,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.1010,2124.10,2338.63,SELL,2025-04-25 15:35:00+05:30,7164.400000,Close Long,-112.5773,...,2025-04-25,10,4,17,2025,0,0,2124.10,2338.63,-0.048138
211221,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.1010,423.40,466.16,SELL,2025-04-25 15:35:00+05:30,5040.300000,Close Long,-22.4402,...,2025-04-25,10,4,17,2025,0,0,423.40,466.16,-0.048138
211222,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,FARTCOIN,1.1010,3599.80,3963.38,SELL,2025-04-25 15:35:00+05:30,4616.900000,Close Long,-190.7894,...,2025-04-25,10,4,17,2025,0,0,3599.80,3963.38,-0.048138


# Save the cleaned dataset

In [13]:
trades.to_csv('../data/preprocessed/historical_data/cleaned_historical_data.csv', index=False)